<a href="https://colab.research.google.com/github/SamyarZamani/mobile-price-prediction/blob/main/mobile_price_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing libreries

In [ ]:
!pip install xgboost

In [ ]:
# اضافه کردن کتاب خانه های مورد نیاز
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# loading dataset

In [ ]:
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [ ]:
df = pd.read_csv("train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1


# data information

In [ ]:
#بررسی مقادیر خالی و نوع داده‌ها
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [ ]:
# price_range مکسیموم و مینوموم فیوچر
print(df['price_range'].max())
print(df['price_range'].min())

3
0


In [ ]:
# ستون و سطر دیتا
df.shape

(2000, 21)

# preprpcessing

In [ ]:
#تقسیم داده برای آموزش و تست
X = df.drop("price_range", axis=1)
y = df["price_range"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# id حذف ستون
test_df = test_df.drop(columns=["id"])

# model train and prediction

In [ ]:
# xgboost اجرا مدل با
# validation پیشبینی مدل با دیتای
# گرفتن دقت مدل
xgb_model = XGBClassifier(random_state=42, eval_metric="mlogloss")
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_val)

print("XGBoost Accuracy:", accuracy_score(y_val, y_pred_xgb))
print(classification_report(y_val, y_pred_xgb))

XGBoost Accuracy: 0.905
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       105
           1       0.88      0.93      0.90        91
           2       0.84      0.85      0.84        92
           3       0.93      0.88      0.91       112

    accuracy                           0.91       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.91      0.91      0.91       400



In [ ]:
# ذخیره مدل
pickle.dump(xgb_model, open("mobile_price_xgb.pkl", "wb"))

# test file pridiction

In [ ]:
# پیشبینی با دیتای تست و ذخیره آن
preds = xgb_model.predict(test_df)

submission = pd.DataFrame({
    "id": test_df.index,
    "price_range": preds
})

submission.to_csv("submission_xgb.csv", index=False)

In [ ]:
# خروجی دیتای تست
submission

,id,price_range
0,0,3
1,1,3
2,2,2
3,3,3
4,4,1
...,...,...
995,995,2
996,996,1
997,997,0
998,998,2
